# replicate the sine model to check if we can deploy the model onto the MCU

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset

import numpy as np
import matplotlib.pyplot as plt
import math

### check if mps support is available -metal performance shaders

In [ ]:
torch.backends.mps.is_available()

In [ ]:
torch.backends.mps.is_built()

In [ ]:
# we can use it by setting the mps device
mps = torch.device("mps")
cpu = torch.device("cpu")

## create data 

In [ ]:
SAMPLES = 1000
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
# generate random numbers
x_values = np.random.uniform(low=0, high=2*math.pi, size=SAMPLES)

# shuffle the values
np.random.shuffle(x_values)

# calculate corresponding y values
y_values = np.sin(x_values)
plt.plot(x_values, y_values, 'b.')
plt.show()

In [ ]:
# add noise
y_values += 0.1 * np.random.randn(y_values.shape[0])

plt.plot(x_values, y_values, 'b.')
plt.show()

In [ ]:
TRAIN_SPLIT = int(0.6 * SAMPLES)
TEST_SPLIT = int(0.2 * SAMPLES + TRAIN_SPLIT)

x_train, x_validate, x_test = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
y_train, y_validate, y_test = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

# create data

In [ ]:
SAMPLES = 1000
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
# generate random numbers
x_values = np.random.uniform(low=0, high=2*math.pi, size=SAMPLES)

# shuffle the values
np.random.shuffle(x_values)

# calculate corresponding y values
y_values = np.sin(x_values)

In [ ]:
# add noise
y_values += 0.1 * np.random.randn(y_values.shape[0])

plt.plot(x_values, y_values, 'b.')
plt.show()

In [ ]:
TRAIN_SPLIT = int(0.6 * SAMPLES)
TEST_SPLIT = int(0.2 * SAMPLES + TRAIN_SPLIT)

x_train, x_validate, x_test = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
y_train, y_validate, y_test = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

In [ ]:
# Plot the data in each partition in different colors:
plt.plot(x_train, y_train, 'b.', label="Train")
plt.plot(x_validate, y_validate, 'y.', label="Validate")
plt.plot(x_test, y_test, 'r.', label="Test")
plt.legend()
plt.show()

# model


In [ ]:
# shape should resemble the one used in the tensorflow tutorial

class Mlp(nn.Module):
    def __init__(self):
        super(Mlp, self).__init__()
        # input layer
        self.fc1 = nn.Linear(1, 16)
        # hidden layer
        self.fc2 = nn.Linear(16, 16)
        # output layer, no activation
        self.fc3 = nn.Linear(16, 1)
    
    def forward(self, x):
        # forward loop to propagate through the network
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## implement dataset and data loader

In [ ]:
class SineDataset(Dataset):
    
    def __init__(self, data):
         # data is tuple of input and target
        self.x_data = data[0]
        self.y_data = data[1]
        assert len(self.x_data) == len(self.y_data)
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        # transform data to tensor and unsqueeze to fit the dimensions
        example = torch.unsqueeze(torch.tensor(self.x_data[idx], dtype=torch.float), dim=0)
        target = torch.unsqueeze(torch.tensor(self.y_data[idx], dtype=torch.float), dim=0)
        
        return example, target

In [ ]:
def train_with_loader(model, loader, opti, crit, device):
    
    model.to(device)
    
    epoch_train_loss = list()

    for _data in loader:
        # send data to device
        _inpt = _data[0].to(device)
        _trgt = _data[1].to(device)

        # make prediction
        _otp = model(_inpt)
        # compute loss
        loss = crit(_otp.to(device), _trgt).requires_grad_(True)
        # zero out gradients
        opti.zero_grad()
        # backward pass
        loss.backward()       
        # optimization step
        opti.step()

        epoch_train_loss.append(loss.detach().cpu().numpy())

    return model, np.average(epoch_train_loss)

In [ ]:
def validate_with_loader(model, loader, device, crit):

    model.to(device)

    val_loss = list()

    for _data in loader:
        # send data to device
        _inpt = _data[0].to(device)
        _trgt = _data[1].to(device)

        # make prediction
        _otp = model(_inpt)
        # compute loss
        loss = crit(_otp.to(device), _trgt)

        val_loss.append(loss.detach().cpu().numpy())
    
    return np.average(val_loss)

In [ ]:
EPOCHS = 100
LR = 0.001
BATCH_SIZE = 1
DEVICE = mps

criterion = nn.MSELoss()

# init model and send to desired device
mlp_3 = Mlp()

optimizer = torch.optim.RMSprop(mlp_3.parameters(), lr=LR)

# init datasets
train_ds = SineDataset((x_train, y_train))
val_ds = SineDataset((x_validate, y_validate))

# dataloaders
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
train_loss = list()
val_loss = list()

for epoch in range(EPOCHS):
    # training step
    mlp_3.train()
    mlp_3, t_loss_epoch = train_with_loader(mlp_3, train_loader, optimizer, criterion, DEVICE)
    train_loss.append(t_loss_epoch)
    
    # validation step
    mlp_3.eval()
    v_loss_epoch = validate_with_loader(mlp_3, val_loader, DEVICE, criterion)
    val_loss.append(v_loss_epoch)
    
    print(f"EPOCH {epoch+1}: Training loss is {t_loss_epoch} - Validation loss is {v_loss_epoch}")

In [ ]:
plt.plot(range(EPOCHS),train_loss, label='Train loss')
plt.plot(range(EPOCHS), val_loss, label='Validation loss')
plt.legend()
plt.show()

In [ ]:
test_ds = SineDataset((x_test, y_test))
test_loader = DataLoader(test_ds, batch_size=10, shuffle=False)

test_loss = validate_with_loader(mlp_3, test_loader, DEVICE, criterion)

print(test_loss)

cpu, 1, 9.35s
cpu, 30, 9.5s
cpu, 50, 9.66s
cpu, 100, 9.27s
cpu, 200, 

mps, 1, 
mps, 30, 
mps, 50,
mps, 100,
mps, 200, 